In [ ]:
import pandas as pd
import numpy as np
import traceback
import datetime
import sys
import locale

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_columns', None)

# Importar Datasets

In [ ]:
def import_data():
    path = "Datasets/"#diretório dos datasets
    
    # --------- as linhas abaixo servem para gerar o nome dos arquivos automaticamente
    prefix = "ABONOP_"#prefixo comun a todos os arquivos
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']#meses presentes
    years = ["2017", "2018", "2019", "2020"]#anos presentes
    ext = ".csv"#extensão dos arquivos
    
    # ------------------------
    
    columns = ["Nome", "CPF", "Descrição do cargo emprego", "Nível de Escolaridade", "Denominação do órgão de atuação",
              "UF da UPAG de vinculação", "Denominação unidade organizacional", "UF da Residência",
              "Cidade da residência", "Situação servidor", "Quantidade de anos no Serviço público",
              "Quantidade de meses no Serviço público","Ano/Mês inicial DO ABONO DE PERMANENCIA","VAL"] #colunas presentes
    
    date_columns = ["Ano/Mês inicial DO ABONO DE PERMANENCIA"]#colunas que devem ser transformadas em datas, porém no dataset estão como inteiros
    float_columns = {"VAL":np.float64}#coluna númerica
    
    datasets = []#esta variavel vai ser usada como uma lista de DataFrames importados
    sep = ";"#separador utilizado nos arquivos
    decimal = "."#valores decimais são separados por virgula
    try:
        name = ""
        for i in years:# esse loop vai importar os datasets ano a ano
            for j in months:# esse loop vai importar os datasets mês a mês
                name = path + prefix + j + i + ext# concatena as informações do arquivo para formar seu nome

                df = pd.read_csv(name, sep =sep, #low_memory = False,
                                names = columns, header = 0,
                                #warn_bad_lines = True
                                encoding = 'cp1252', index_col=False)#ver documentação do pandas
                date = datetime.date(int(i), int(j), 1)#cria uma data do tipo 1/j/i
                df["DATE"] = [date for k in range(df.shape[0])]#adiciona uma coluna com a data
                datasets.append(df)
    except Exception as e:#ocorreu algum exceção
        traceback.print_exc()#printa a pilha de execução
        print("Erro na planilha: {}".format(name))#printa a plainlha que ocorreu o erro
    finally:
        if len(datasets) > 0:
            datasets = pd.concat(datasets)#concatena os datasets que foram importados com sucesso
            datasets.drop_duplicates(subset = columns, inplace = True)#tira as colunas duplicadas, não considera a coluna DATE
            
            datasets.reset_index(inplace = True)
            datasets.drop(columns = ["index"], inplace = True)
            
        return datasets

In [ ]:
def import_dataset(name, sep = ",", columns = []):
    df = pd.read_csv(name, sep =sep, #low_memory = False,
                    names = columns, header = 0,
                    #warn_bad_lines = True
                   index_col=False)#ver documentação do pandas
    return df

In [ ]:
abono_dataset = import_data()

In [ ]:
cidades_dataset = import_dataset("Datasets/municipios.csv", sep = ";", columns = ["ConcatUF+Mun", "IBGE", "IBGE7", "UF", "Município", 
                                               "Região", "População 2010", "Porte", "Capital"])

# Retirar espaços em branco em excesso e alguns ajustes

In [ ]:
def clean_spaces(df, text_columns, number_columns):
    #retira os espaços em branco e coloca todos os caracteres em caixa baixa
    df = df.applymap(
        lambda x: " ".join( x.split() ).lower() if isinstance(x, str) else x, 
        na_action = 'ignore')
    #troca todas as ',' por '.'
    df[number_columns] = df[number_columns].applymap(
        lambda x: ".".join(x.split(",")) if isinstance(x, str) else x, 
        na_action = 'ignore')
    
    return df

In [ ]:
text_columns = ["Nome", "CPF", "Descrição do cargo emprego", "Nível de Escolaridade", "Denominação do órgão de atuação",
              "UF da UPAG de vinculação", "Denominação unidade organizacional", "UF da Residência",
              "Cidade da residência", "Situação servidor"]
number_columns = ["Quantidade de anos no Serviço público","Quantidade de meses no Serviço público",
                  "Ano/Mês inicial DO ABONO DE PERMANENCIA","VAL"]
abono_dataset = clean_spaces(abono_dataset, text_columns, number_columns)

In [ ]:
dtypes = {
    "Quantidade de anos no Serviço público": int,
    "Quantidade de meses no Serviço público": int,
    "VAL": np.float64
}
abono_dataset = abono_dataset.astype(dtypes)

# Corrigir coluna Cidade da residência

In [ ]:
def distancia_hamming(string1, string2):
    """
    Calcula a distancia de hamming entre as strings de entrada
    """
    #código de https://stackoverflow.com/questions/54172831/hamming-distance-between-two-strings-in-python
    return sum(c1 != c2 for c1, c2 in zip(string1, string2))

In [ ]:
def corrige_cidade(cidades_do_estado , cidade):
    """
    Essa função corrige a cidade dada por uma na lista cidades_do_estado
    A função de distancia de hamming é utilizada como critério para escolha
    O algoritmo escolhe o elemento com menor distância de hamming da lista cidades_do_estado
    com cidade
    """
    if cidade in cidades_do_estado:#verifica se a cidade já existe na lista
        return cidade
    else:#se não ela vai ser processada
        cid_min = ""#inicializa a cidade com distancia de hamming minima
        distancia_min = 999999#inicializa a minima distancia de hamming encontrada
        for cid in cidades_do_estado:#percorre todas as cidades do estado
            distancia = distancia_hamming(cid, cidade)#calcula a distancia entre a cidade atual e a cidade informada
            if distancia < distancia_min:
                #se essa distancia for menor que a minima até agora
                #então é feita uma substituição
                cid_min = cid
                distancia_min = distancia
        return cid_min#retorna a cidade com distancia minima encontrada

In [ ]:
estados = set(cidades_dataset["UF"].to_list())
cidades_por_estado = dict()
for estado in estados:#organiza um diciionário de cidades por estado
    cds = cidades_dataset.loc[cidades_dataset["UF"] == estado]["Município"].to_list()
    cidades_por_estado[estado] = list(map(lambda x: x.lower(), cds))

In [ ]:
#realiza correção das cidades
idx_cidade = 9#indice da coluna cidade
idx_uf = 8#indice da coluna uf
mudancas = []#esse array vai salvando as correçoes de cada linha
cache = dict()#dicionario de cache para poupar processamento
for row in abono_dataset.itertuples():
    uf = row[idx_uf].upper()#transforma em caixa alta pois as chaves em cidades_por_estado estão em caixa alta, 
    if uf in cidades_por_estado:# se a uf da linha existe no diciionario de cidades_por_estado
        
        cd_desconhecida = row[idx_cidade]
        cidade = ""
        if cd_desconhecida in cache:#verifica se a cidade já foi processada
            cidade = cache[cd_desconhecida]#recupera o resultado
        else:#se não foi
            cidades = cidades_por_estado[uf]#obtem as cidades do estado daquela linha
            cidade = corrige_cidade(cidades, cd_desconhecida)#corrige a cidade
            cache[cd_desconhecida] = cidade#salva o resultado na cache
        mudancas.append(cidade)#adiciona na lista de mudancas
    else:# se não existe não faz nenhum processamento
        mudancas.append(row[idx_cidade])
        #print(row[idx_cidade], cidade)
        
abono_dataset["Cidade da residência"] = pd.Series(data = mudancas)

# Corrigir coluna Nível de Escolaridade

In [ ]:
#realiza correção do nível de escolaridade
idx_escolaridade = 4#indice da coluna Nível de Escolaridade

mud_escolaridade = {
    "segundo grau incompleto": "ensino medio incompleto",
    "4a. serie do primeiro grau completa": "ensino fundamental incompleto",
    "primeiro grau incomp.-ate a 4a.serie incomp.": "ensino fundamental incompleto"
}
mudancas =[]
for row in abono_dataset.itertuples():
    if row[idx_escolaridade] in mud_escolaridade:
        mudancas.append(mud_escolaridade[row[idx_escolaridade]])
    else:
         mudancas.append(row[idx_escolaridade])
abono_dataset["Nível de Escolaridade"] = pd.Series(data = mudancas)

# Testes

In [ ]:
abono_dataset